In [ ]:
library(pheatmap)
library(ggplot2)
library(ggrepel)
library(umap)
library(Rtsne)
library(cluster)
set.seed(5122721)

cbPalette <- c("#006400","#ff4500","#ffd700","#11ff04","#12fefe","#1113ff","#1e90ff","#ff69b4")
names(cbPalette) <- c("A","B","C","D","E","F","G","others")

data <- read.csv("use_abs_sites.csv", header = T, row.names=1)
data[data>5] <- 5
sites <- c('K417','E484','N501','G446','N440','G339','Q493','G496','Q498','S371','S373','S375','S477','T478','Y505')
data <- t(data)

abanno <- read.csv("list.txt", row.names=1)

use_sites <- rownames(data[rowSums(data>1.0)>=6,])
data <- data[use_sites,]
colnames(data)[colSums(data)==0]
length(use_sites)

dis <- 1-cor(data, method='pearson')
mdsc <- cmdscale(dis, k=6)
mds <- Rtsne(mdsc)$Y
rownames(mds) <- colnames(data)

um <- as.data.frame(mds)
colnames(um) <- c('MDS1','MDS2')
um$name <- rownames(um)
um$show_name <- abanno[um$name, 'show_name']
labels = pam(mdsc, k=6)$clustering

um$cluster <- labels
abanno <- read.csv("list.txt", header = T)
rownames(abanno) <- abanno$name
abanno <- abanno[,-c(1)]
um$mode <- abanno[um$name, 'mode']
um$mode[is.na(um$mode)] <- "others"
um$ic50 <- abanno[um$name, 'IC50']
um$betabind <- abanno[um$name, 'BetaBLI']
um$Class <- c("D","B","C","A","F","E")[um$cluster]
dev.off()
pdf("abs_heatmap.pdf", height=24, width=24)
show_dis = 1-as.matrix(dis)
rownames(show_dis) <- abanno[rownames(show_dis), 'show_name']
colnames(show_dis) <- abanno[colnames(show_dis), 'show_name']
pheatmap(show_dis, border=F)
dev.off()

sites <- c('K417','E484','N501','G446','N440','G339','Q493','G496','Q498','S371','S373','S375','S477','T478','Y505')
sitesdata <- read.csv("use_abs_sites.csv", header = T, row.names=1)
mutdata <- read.csv("use_abs_muts_concern.csv", header=T, row.names=1)
um$K417N <- mutdata[rownames(um),'K417N']
um$K417NQ493R <- mutdata[rownames(um),'K417N']+mutdata[rownames(um),'Q493R']
um$E484A <- mutdata[rownames(um),'E484A']
um$N440KG446S <- mutdata[rownames(um),'G446S']+mutdata[rownames(um),'N440K']
um$G339D <- mutdata[rownames(um),'G339D']
um$S371LS373PS375F <- mutdata[rownames(um),'S371L']+mutdata[rownames(um),'S373P']+mutdata[rownames(um),'S375F']

um$total_escape_O <- rowSums(mutdata)[rownames(um)]
um$escape <- 0
um[rowSums(mutdata>0.2)>0,'escape'] <- 1
um$FC_Omicron <- abanno[um$name, "FC_Omicron"]
um$sarsic50 <- as.numeric(abanno[um$name, 'SARS_IC50_raw'])
um$sarsic50[is.na(um$sarsic50)] <- 100
um$sarsic50[um$sarsic50>10] <- 100

um$betaic50 <- as.numeric(abanno[um$name, 'Beta_IC50_raw'])
um$betaic50[is.na(um$betaic50)] <- 100
um$betaic50[um$betaic50>10] <- 100

um$omic50 <- as.numeric(abanno[um$name, 'Omicron_IC50_raw'])
um$omic50[is.na(um$omic50)] <- 100
um$omic50[um$omic50>10] <- 100

um$D614Gic50 <- as.numeric(abanno[um$name, 'D614G_IC50_raw'])
um$D614Gic50[is.na(um$D614Gic50)] <- 100
um$D614Gic50[um$D614Gic50>10] <- 100

pdf("projection.pdf", height=18,width=18)
ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=Class), size=5, shape=21, alpha=0.7,color="black")+
    geom_text_repel(aes(label=show_name),max.overlaps = 30)+
    theme_classic()+
    theme(aspect.ratio = 1, text = element_text(size = 20)) + xlab("TSNE 1")+ylab("TSNE 2")


ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=mode), size=5, shape=21, alpha=0.7,color="black")+
    geom_text_repel(aes(label=show_name),max.overlaps = 30)+
    theme_classic()+labs(fill='Binding Mode')+
    scale_fill_manual(values=cbPalette)+
    theme(aspect.ratio = 1,text = element_text(size = 20)) + xlab("TSNE 1")+ylab("TSNE 2")+guides(fill = guide_legend(override.aes = list(alpha=1)))

ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=sarsic50), size=3, shape=21, alpha=0.6,color="black")+
    geom_text_repel(aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+
    # scale_fill_gradient2(low="red",mid="#FF8989",high="#ffffff", midpoint=0.1, oob=squish, limits=c(0.01,10), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    scale_fill_gradient(low="red",high="#ffffff", oob=squish, limits=c(0.01,10), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2")

show_abs = c('REGN10933','REGN10987','CB6','LY-CoV555','BD30-604','BD30-593','S309','BRII-196','AZD8895','AZD1061','S304')
dev.off()
pdf("projection_s.pdf", height=5, width=5)
ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=Class), size=4, shape=21, alpha=0.5,color="black")+
    # scale_fill_manual(values=c("#fe6f5e", "#87a96b", "#007fff"), breaks = factor(c("Class I", "Class II", "Class III/IV")))+
    geom_text_repel(data=um[show_abs,], aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2") + theme(legend.title = element_blank())

ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=mode), size=4, shape=21, alpha=0.5,color="black", show.legend = FALSE)+
    geom_text_repel(data=um[show_abs,], aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+labs(fill='Binding Mode')+
    scale_fill_manual(values=cbPalette)+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2")+guides(fill = guide_legend(override.aes = list(alpha=1))) + theme(legend.title = element_blank())

# evaluate escape

library(scales)
ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=mode), size=4, shape=21, alpha=0.3,color="black")+geom_point(data=um[rowSums(mutdata>0.2)>0,], color="black",shape=21,size=4,alpha=0.7,fill="black")+
    geom_text_repel(data=um[show_abs,], aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+labs(fill='Binding Mode')+
    scale_fill_manual(values=cbPalette)+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2")+guides(fill = guide_legend(override.aes = list(alpha=1))) + theme(legend.title = element_blank())

ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=total_escape_O), size=3, shape=21, alpha=0.5,color="black", show.legend = FALSE)+
    geom_text_repel(data=um[show_abs,], aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+
    scale_fill_gradient(low="#eeeeee",high="red", limits=c(0,0.3), oob=squish)+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2") + theme(legend.title = element_blank())

ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=FC_Omicron), size=3, shape=21, alpha=0.6,color="black", show.legend = FALSE)+
    geom_text_repel(data=um[show_abs,], aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+
    scale_fill_gradient(low="#ffffff",high="red", oob=squish, limits=c(8,40), trans="log")+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2")
ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=FC_Omicron), size=3, shape=21, alpha=0.6,color="black")+
    geom_text_repel(data=um[show_abs,], aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+
    scale_fill_gradient(low="#ffffff",high="red", oob=squish, limits=c(8,40), trans="log", breaks=c(5,10,20,40))+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2")

ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=omic50), size=3, shape=21, alpha=0.6,color="black")+
    geom_text_repel(data=um[show_abs,], aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+
    # scale_fill_gradient2(low="red",mid="#FF8989",high="#ffffff", midpoint=0.1, oob=squish, limits=c(0.01,10), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    scale_fill_gradient(low="red",high="#ffffff", oob=squish, limits=c(0.01,8), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2")

ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=sarsic50), size=3, shape=21, alpha=0.6,color="black", show.legend = FALSE)+
    geom_text_repel(data=um[show_abs,], aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+
    # scale_fill_gradient2(low="red",mid="#FF8989",high="#ffffff", midpoint=0.1, oob=squish, limits=c(0.01,10), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    scale_fill_gradient(low="#d52a2a",high="#ffffff", oob=squish, limits=c(0.01,8), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2")
ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=sarsic50), size=3, shape=21, alpha=0.6,color="black")+
    geom_text_repel(data=um[show_abs,], aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+
    # scale_fill_gradient2(low="red",mid="#FF8989",high="#ffffff", midpoint=0.1, oob=squish, limits=c(0.01,10), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    scale_fill_gradient(low="#d52a2a",high="#ffffff", oob=squish, limits=c(0.01,8), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2")
ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=betaic50), size=3, shape=21, alpha=0.6,color="black")+
    geom_text_repel(data=um[show_abs,], aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+
    # scale_fill_gradient2(low="red",mid="#FF8989", high="#ffffff", midpoint=0.1, oob=squish, limits=c(0.01,10), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    scale_fill_gradient(low="red", high="#ffffff", oob=squish, limits=c(0.01,8), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2")
ggplot(um, aes(x=MDS1,y=MDS2))+
    geom_point(aes(fill=D614Gic50), size=3, shape=21, alpha=0.6,color="black")+
    geom_text_repel(data=um[show_abs,], aes(label=show_name),max.overlaps = 20,fontface = "bold")+
    theme_classic()+
    # scale_fill_gradient2(low="red",mid="#FF8989", high="#ffffff", midpoint=0.1, oob=squish, limits=c(0.01,10), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    scale_fill_gradient(low="red", high="#ffffff", oob=squish, limits=c(0.01,8), trans="log", breaks=c(0.01, 0.1, 1, 10))+
    theme(aspect.ratio = 1) + xlab("TSNE 1")+ylab("TSNE 2")

dev.off()
write.csv(um[,c("mode","Class")], file="new_clustering.csv", quote=F)

In [ ]:
# binarized escape heatmap

cbPalette <- c("#006400","#ff4500","#ffd700","#11ff04","#12fefe","#1113ff","#1e90ff","#ff69b4")
names(cbPalette) <- c("A","B","C","D","E","F","G","others")
library(ComplexHeatmap)
library(dplyr)
library(circlize)
data <- read.csv("use_abs_sites.csv", header = T, row.names=1)
data[data>5] <- 5
sites <- c('K417','E484','N501','G446','N440','G339','Q493','G496','Q498','S371','S373','S375','S477','T478','Y505')
neighbouring_sites <- as.numeric(substring(sites, 2,4))
neighbouring_sites <- c(neighbouring_sites-1, neighbouring_sites+1)

# neighbouring_sites <- c(neighbouring_sites-2, neighbouring_sites-1, neighbouring_sites+1, neighbouring_sites+2)
neighbouring_sites <- data.frame(site=c(sites, sites), neighbour=neighbouring_sites)
RBD_dict <- data.frame(site=substring(unique(colnames(data)),2,4),wt_site=unique(colnames(data)))
rownames(RBD_dict) <- RBD_dict$site
neighbouring_sites$wt_site <- RBD_dict[as.character(neighbouring_sites$neighbour),"wt_site"]
neighbouring_sites <- na.omit(neighbouring_sites)
nb_escape <- as.data.frame(cbind(neighbouring_sites, t(data[,neighbouring_sites$wt_site]))[,-c(2,3)] %>% group_by(site) %>% summarise_all(sum))
rownames(nb_escape) <- nb_escape$site
nb_escape <- nb_escape[,-1]>1
nb_escape <- rbind(nb_escape, rep(FALSE, ncol(nb_escape))) 
rownames(nb_escape)[nrow(nb_escape)] <- "G496"

data <- t(data)
data_ht <- as.data.frame(data[intersect(rownames(data), sites),])

a <- read.csv("use_abs_muts_concern.csv", header = T, row.names = 1)

abanno <- read.csv("list.txt", row.names=1, header=T)

abanno$escape=(apply(a, 1, max)>0.2)[rownames(abanno)]
abanno <- abanno[colnames(data),]
dismat <- dist(a>0.2, method='binary')
ab_order <- rownames(abanno)

site_order <- c('K417N','E484A','N501Y','Q493R','G446S',
                'N440K','G339D','G496S','Q498R','S371L',
                'S373P','S375F','S477N','T478K','Y505H')

clustercolor <- c("#fe6f5e", "#87a96b", "#007fff","#3d2b1f")
names(clustercolor) <- c(1,2,3,4)

a <- (as.matrix(a)>0.2)*2
col_fn <- colorRamp2(c(-1,0,1,2), c("blue","#eeeeef","#dd8081", "#bf0c00"))
st_order <- c('K417','E484','N501','Q493','G446',
                'N440','G339','G496','Q498','S371',
                'S373','S375','S477','T478','Y505')
a <- a[,site_order]
escape_site <- (data_ht>1.0)[st_order, ]
abanno$escape_site <- apply(escape_site, 2, max)

abanno$escape[(!abanno$escape) & (abanno$escape_site)] <- "SITE"
rownames(escape_site) <- colnames(a)

a <- t(a)+escape_site
a[a>2] <- 2

rownames(nb_escape)
substring(rownames(a),1,4)
nb_escape <- nb_escape[substring(rownames(a),1,4), colnames(a)]
rownames(nb_escape) <- rownames(a)

add_sites <- c('K417', 'D420', 'L455','F456', 'A475','N487','G476','F486',
               'E484', 'F490', 'L452', 
               'K444', 'V445','G446', 'G447', 'N448','Y449',
               'N334','L335','G339','E340','R346', 'V503','G504', 'K378')
data <- as.matrix(read.csv("use_abs_sites.csv", header = T, row.names=1)[,add_sites])
data <- 1.0*(t(data)>1.0)

sites_split <- factor(c(rep('mut', nrow(a)), rep('site', nrow(data))))
ab_order <- hclust(dist(t(a), method="manhattan"))$order

abanno <- abanno[colnames(a),]
colnames(a) <- abanno[colnames(a), 'show_name']

abanno[(abanno$escape =="FALSE") & (apply(a, 2, min)<0),"escape"] <- "NB"
write.table(data.frame(id=abanno$id, name=rownames(abanno),impact=c("TRUE"="Mut","SITE"="Site","FALSE"="No","NB"="Neighbour")[abanno[,'escape']]), row.names=F, file="./impact.txt", quote=F, sep="\t")
a <- a[,use_abs]
abanno <- abanno[use_abs,]

hacol <- ComplexHeatmap::HeatmapAnnotation(
        `Epitope Group` = abanno$mode,
        `Beta IC50 Fold Change` = log10(abanno$FC_Beta), #abanno$Beta_IC50_2 / abanno$D614G_IC50_raw,
        `Omicron IC50 Fold Change` = log10(abanno$FC_Omicron), #abanno$Omicron_IC50_2 / abanno$D614G_IC50_raw,
        `Omicron Binding` = factor(abanno$O.bind>0),
        col = list(
            `Beta IC50 Fold Change` = colorRamp2(c(log10(8),log10(40)), c("blue", "red")),
            `Omicron IC50 Fold Change` = colorRamp2(c(log10(8),log10(40)), c("blue", "red")),
            `Omicron Binding` = c("FALSE"="red", "TRUE"="blue"),
            `Epitope Group` = cbPalette
        ), annotation_name_side = "left", border=T,gap=unit(2,'mm')
)

abanno$mode[is.na(abanno$mode)] <- "others"
dev.off()
pdf("binarized_escape_heatmap.pdf", width=19, height=5)
p<- ComplexHeatmap::Heatmap(
            a, row_names_side = "left", column_names_side = "bottom", column_names_gp = gpar(fontsize=6), column_gap=unit(c(1.5), "mm"), 
            row_gap=unit(c(5),'mm'), #column_order = order(-abanno$FC_Omicron,factor(abanno$escape, levels=c('TRUE','SITE','NB','FALSE'))),
            cluster_columns = T, show_column_dend=F, clustering_distance_rows = "spearman", clustering_method_rows = "single", column_title=NULL,
            cluster_rows=F,top_annotation=hacol, 
            # column_split = factor(paste(abanno$source_main,abanno$mode)), 
            column_split = factor(abanno$mode), 
            row_names_gp = gpar(fontsize=14),
            # row_split = sites_split,
            cluster_column_slices = FALSE, border=T,
            col=col_fn, heatmap_legend_param = list(title = "Impact", at = c(0,1,2,-1), legend_gp = gpar(fill = col_fn(c(0,1,2,-1))),
        labels = c("No Escape", "By site", "By mutation", "By neighbors"), color_bar="discrete")#, rect_gp = gpar(col = "black", lwd = 0.5)
)
draw(p)
dev.off()


In [ ]:
# mode specific heatmaps A

library(ComplexHeatmap)
library(circlize)
ACE2_face <- c(417,445,446,449,453,455,456,473,475,476,484,486,487,489,493,496,498,500,501,502,503,505)
N_sites <- 15
site_sum <- read.csv("use_abs_sites.csv", header=T)
abanno <- read.csv("list.txt", header = T, row.names=1)

col_fun = colorRamp2(c(0, 5), c("#fcfcfc", "red"))
abanno$special_ones[is.na(abanno$special_ones)] <- 10000
abanno$Beta_IC50[abanno$Beta_IC50 == "--"] <- Inf
rownames(site_sum) <- site_sum$antibody
site_sum <- site_sum[,-1]
site_sum[site_sum >=10] <- 10.0

mdd = 'F'

site_sum_f <- na.omit(site_sum[rownames(abanno[abanno$mode == mdd, ]),])

site_sum_f <- site_sum_f[!(rownames(site_sum_f) %in% c("COVOX-253H55L","COVOX-253H165L")),]
rownames(site_sum_f)
N_abs <- min(30, nrow(site_sum_f))

site_order <- data.frame(site=colnames(site_sum_f), prio=0)
rownames(site_order) <- site_order$site
if(mdd == 'D'){site_order['N440', 'prio'] <- 1}
site_sum_f <- as.matrix(site_sum_f[order(abanno[rownames(site_sum_f), "special_ones"], abanno[rownames(site_sum_f),"D614G_IC50_raw"])[1:N_abs], ])
site_sum_f <- as.matrix(site_sum_f[,order(site_order[colnames(site_sum_f),'prio'], colSums(site_sum_f),decreasing=T)[1:(N_sites)]])

row_split <- factor(abanno[rownames(site_sum_f), 'OmicronBinding'],levels=c(0,1))

abanno$Omicron_IC50 <- as.numeric(abanno$Omicron_IC50_raw)
abanno$Omicron_IC50[is.na(abanno$Omicron_IC50)] <- Inf

abanno$Beta_IC50 <- as.numeric(abanno$Beta_IC50_raw)
abanno$Beta_IC50[is.na(abanno$Beta_IC50)] <- Inf

hacol <- ComplexHeatmap::HeatmapAnnotation(
        `ACE2 Interface`=as.numeric(substring(colnames(site_sum_f), 2,4)) %in% ACE2_face,
        col = list(
            `ACE2 Interface` = c("TRUE" = "#e32636", "FALSE"= "#eeeeee")
        ), annotation_name_side = "left",annotation_name_gp=gpar(fontsize=6,fontface="bold"),
        border=TRUE,simple_anno_size = unit(3, "mm")
)
harow <- ComplexHeatmap::rowAnnotation(
        `Omicron IC50`=log10(abanno[rownames(site_sum_f), 'Omicron_IC50']),
        `Omicron IC50\nFold Change`=log10(abanno[rownames(site_sum_f),'FC_Omicron']),
        # `Omicron IC50 FC`=abanno[rownames(site_sum_f),'FC_Omicron'],
        `Beta IC50\nFold Change`=log10(abanno[rownames(site_sum_f),'FC_Beta']),
        # `Beta IC50`=log10(abanno[rownames(site_sum_f), 'Beta_IC50']),
        col = list(
            `Omicron IC50`=colorRamp2(c(-2,-1,1), c("blue", "#6100DD","red")),
            `Omicron IC50\nFold Change` = colorRamp2(c(0.903,1.602), c("blue", "red")),
            # `Omicron IC50 FC` = colorRamp2(c(5,40), c("blue", "red")),
            `Beta IC50\nFold Change`=colorRamp2(c(0.903,1.602), c("blue", "red"))
            # `Beta IC50` = colorRamp2(c(-3,2), c("blue","red"))
        ), annotation_name_side = "top", annotation_name_gp=gpar(fontsize=7,fontface="bold"),
        annotation_legend_param=list(
            `Omicron IC50\nFold Change` = list(at=c(0.699,1,1.301,1.602)),
            `Beta IC50\nFold Change` = list(at=c(0.699,1,1.301,1.602))
        ),
        border=TRUE, gap=unit(1,'mm'),simple_anno_size = unit(4, "mm"))
dev.off()
nrow(site_sum_f)
row_split <- rep(1, nrow(site_sum_f))

sites <- c('K417','E484','N501','G446','N440','G339','Q493','G496','Q498','S371','S373','S375','S477','T478','Y505')
col_split <- rep(1, ncol(site_sum_f))
names(col_split) <- colnames(site_sum_f)
col_split[intersect(names(col_split),sites)] <- 0
if (mdd=="F"){
    row_split[site_sum_f[,"G504"]>2] <- 0
    col_split[c('V503','G504')] <- 0.5
}

col_names_col <- rep("black", ncol(site_sum_f))
names(col_names_col) <- colnames(site_sum_f)
col_names_col[intersect(names(col_names_col), sites)] <- '#cf0c00'

pdf(paste(mdd,"_mode_esc_hm.pdf",sep=""), height=8, width=6)
rownames(site_sum_f) <- abanno[rownames(site_sum_f), "show_name"]
p <- ComplexHeatmap::Heatmap(site_sum_f, row_names_side = "left", column_names_side = "top", clustering_distance_rows="manhattan",
             clustering_method_rows="single",
             name="escape_score", #row_order=order(abanno[rownames(site_sum_f), mode_site[mdd]]),
             col=col_fun, #rect_gp = gpar(col = "#666666", lwd = 0.5), #
             show_column_dend = F, show_row_dend = F, row_split = row_split, cluster_row_slices=F, row_gap=unit(0,'mm'), row_title=NULL,
             row_names_gp=gpar(fontsize=6,fontface="bold"), column_split=col_split, column_gap=unit(0,'mm'),column_title=NULL,
             column_names_gp=gpar(fontsize=7, fontface="bold", col=col_names_col), cluster_column_slices=F,
             # row_order=row_order,
             bottom_annotation = hacol, right_annotation = harow,width = unit(4, "cm"), height = unit(nrow(site_sum_f)/5, "cm"))

print(p)
dev.off()

In [5]:
# EUA Abs IC50 heatmap
library(ComplexHeatmap)
library(circlize)

data <- as.matrix(read.table("eua_abs_ic50.txt", sep="\t", header =T, row.names=1))
abanno <- read.csv("list.txt", row.names=1,header =T)
rownames(data) <- abanno[rownames(data), 'show_name']
# data <- log10(data)
data[data>10] <- 100

dev.off()
pdf("eua_abs.pdf",height=8,width=5)
p <- Heatmap(data, show_row_dend=F, show_column_dend=F,
        col=colorRamp2(c(0,10), c("#efefef","#dc4154")),row_names_side="left",column_names_side="top",
        rect_gp = gpar(col = "#434343", lwd = 1),width = unit(7, "cm"), height = unit(9, "cm"),
        cluster_rows=F,cluster_columns=F,
        cell_fun = function(j, i, x, y, width, height, fill) {
            if (pindex(data, i, j) > 10) {
                grid.text(sprintf(">10"), x,y,gp=gpar(fontsize=10))
            }
            else{
            grid.text(sprintf("%.3f", pindex(data, i, j)), x, y, gp = gpar(fontsize = 10))
        }
            }
       )
draw(p)
dev.off()


circlize version 0.4.12
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: https://jokergoo.github.io/circlize_book/book/

If you use it in published research, please cite:
Gu, Z. circlize implements and enhances circular visualization
  in R. Bioinformatics 2014.

This message can be suppressed by:
  suppressPackageStartupMessages(library(circlize))




null device 
          1

null device 
          1

In [ ]:
#K417N
abanno <- read.csv("list.txt", header=T,row.names=1)
data <- read.csv("use_abs_muts_concern_ext.csv", header=T, row.names=1)

abanno[abanno$FC.417N == ">100","FC.417N"] <- 100
abanno$FC.417N <- as.numeric(abanno$FC.417N)
use_abs <- rownames(abanno[!is.na(abanno$FC.417N) & abanno$mode == "A",])

plot_mat <- as.matrix(data[use_abs, "K417N"])
colnames(plot_mat) <- c("K417N Escape Score")
rownames(plot_mat) <- use_abs

col_fun = colorRamp2(c(0, 0.25, 0.5), c("blue","#df0049","red"))
# elisa_fn <- colorRamp2(c(0,3),c("red","blue"))
row_anno <- rowAnnotation(
    K417NIC50FC=log10(abanno[use_abs,"FC.417N"]),
    # beta_ELISA_OD=as.numeric(abanno[use_abs,"beta_ELISA_OD"]),
    Epitope=abanno[rownames(plot_mat), 'mode'],
    col=list(
        Epitope=cbPalette,
        K417NIC50FC=colorRamp2(c(0.903,1.602),c("blue","red"))
        # beta_ELISA_OD=elisa_fn
    ), annotation_name_side="top",border=T,
    annotation_legend_param=list(
            K417NIC50FC = list(at=c(0.699,1,1.301,1.602))
        )
)

row_order <- order(plot_mat[,1])
rownames(plot_mat) <- abanno[rownames(plot_mat), 'show_name']
dev.off()
pdf("K417NFC.pdf", width=2.8,height=8)
p <- Heatmap(plot_mat, col = col_fun, row_order = row_order, 
             column_names_side = "top",
             right_annotation=row_anno)
draw(p)
dev.off()

In [ ]:
#K484K/A
abanno <- read.csv("list.txt", header=T,row.names=1)
data <- read.csv("use_abs_muts_concern_ext.csv", header=T, row.names=1)

abanno[abanno$FC.484K == ">100","FC.484K"] <- 100
abanno$FC.484K <- as.numeric(abanno$FC.484K)
use_abs <- rownames(abanno[!is.na(abanno$FC.484K) & abanno$mode == "C",])

plot_mat <- as.matrix(data[use_abs, c("E484A","E484K")])
colnames(plot_mat) <- c("E484A Escape Score", "E484K Escape Score")
rownames(plot_mat) <- use_abs

col_fun = colorRamp2(c(0, 0.25, 0.5), c("blue","#df0049","red"))
row_anno <- rowAnnotation(
    E484KIC50FC=abanno[use_abs,"FC.484K"],
    Epitope=abanno[use_abs, 'mode'],
    col=list(
        Epitope=cbPalette,
        E484KIC50FC=colorRamp2(c(0.903,1.602),c("blue","red"))
    ), annotation_name_side="top",border=T
)

row_order <- order(plot_mat[,1],abanno[use_abs,"FC.484K"])
rownames(plot_mat) <- abanno[rownames(plot_mat), 'show_name']
dev.off()
pdf("E484KFC.pdf", width=3,height=8)
p <- Heatmap(plot_mat, col = col_fun, row_order = row_order, 
             column_names_side = "top", show_column_dend=F, column_order=c(1,2),
             right_annotation=row_anno)
draw(p)
dev.off()